In [ ]:
!pip install cohere
!pip install openai
!pip install tiktoken

In [ ]:
import pandas as pd
import os
import json
import time
import string
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [ ]:
def batch_gpt(message_prompt):
    responses = []
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[{"role": "user", "content": message_prompt}]
    )
    r = response.choices[0].message.content 
    responses.append(r)

    return r, responses

In [ ]:
def get_mess():
  mess = """
      You need to determine the smallest possible elementary discourse units within sentences in a given text by breaking down each sentence into its smallest elementary discourse units.
      Do not consider a whole sentence as one elementary discourse unit. Use the punctuation, morphological, lexical, and syntactic features to divide each sentence into small units. Return each elementary discourse unit assigned with a unique id, without any additional explanations, in the following format:
      {
      "elementary_discourse_units": [{"edu": "edu_1"}, {"edu": "edu_2"}, ..., {"edu": "edu_n"}]
      }
      Text example:
      """

  return mess

In [ ]:
dataset_with_texts = pd.read_csv('local_path/eng_dep_covdtb_texts_test.csv')
dataset_with_edus = pd.read_csv('/local_path/eng_dep_covdtb_edus_test.csv')

In [ ]:
def get_edu_list(ID):

  return dataset_with_edus[dataset_with_edus['Document ID'] == ID]['Document EDUs']

In [ ]:
final_result = []
for i in range(len(dataset_with_texts)):
  doc_id = dataset_with_texts.iloc[i][0]
  document = dataset_with_texts.iloc[i][1]
  real_edus = get_edu_list(doc_id)[i]
  m = str(str(get_mess()) + str(document))
  result, response = batch_gpt(m)
  count = 0
  correctly_segmented = []
  segments = []
  result = result.replace('json', '')
  result = result.replace('`', '')
  result = result.strip()
  res = (json.loads(result))['elementary_discourse_units']
  for i in range(len(res)):
    cleaned_segment = res[i]['edu'].replace("<S>", "").replace("<P>", "").strip(string.whitespace + string.punctuation)
    segments.append(cleaned_segment)
    if cleaned_segment in real_edus:
      correctly_segmented.append(cleaned_segment)
      count += 1
  dict_example = {'text number' : doc_id, "real total units" : len(real_edus), "chatgpt total units" : len(res), "chatgpt correctly segmented units" : count, "segments" : segments, "correct segments" : correctly_segmented}
  final_result.append(dict_example)
  time.sleep(5)

In [ ]:
units_dataset = pd.DataFrame(final_result)
units_dataset

In [ ]:
units_dataset.to_csv('/local_path/chat_gpt4_discourse_units_identification.csv', index=False)

In [ ]:
import ast

for i in range(0, len(units_dataset)):
  real_edus = get_edu_list(units_dataset.iloc[i]['text number'])
  units_dataset.loc[i, 'real total units'] = len(ast.literal_eval(real_edus[i]))


In [ ]:
suma = 0.0
s1 = 0.0
s2 = 0.0
for i in range(len(units_dataset)):
  print(units_dataset.iloc[i]['text number'])
  print(int(units_dataset.iloc[i]['chatgpt correctly segmented units'])/int(units_dataset.iloc[i]['real total units']))
  suma += int(units_dataset.iloc[i]['chatgpt correctly segmented units'])/int(units_dataset.iloc[i]['real total units'])
  s1 += int(units_dataset.iloc[i]['chatgpt correctly segmented units'])
  s2 += int(units_dataset.iloc[i]['real total units'])
  print("-----------------------------------------------------")
print(s1 / s2)

00c862a6402381a8541bcffe843696e059095300
0.7692307692307693
-----------------------------------------------------
0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83
0.5238095238095238
-----------------------------------------------------
0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4
1.1818181818181819
-----------------------------------------------------
09bdbea1700a3826acc33484ed51e2fa8c23f047
1.3333333333333333
-----------------------------------------------------
0b362bc7e9278b56c2740ab7bc10e91c036f835a
0.4166666666666667
-----------------------------------------------------
0b6936373351a029b06b8784ff8077d755e7ee96
0.7916666666666666
-----------------------------------------------------
0b7581ab8ae5da78a11692b4c7d9399ec6dea283
0.6363636363636364
-----------------------------------------------------
0c44f901baca3850ff6e79e5f26ee898faa9b68d
1.1428571428571428
-----------------------------------------------------
1399e34d62b515e482ccde6c4cae500b19546909
0.6
-------------------------------------------

In [ ]:
correct = 0
total = 0

for i in range(len(units_dataset)):
  total_segments = ast.literal_eval(get_edu_list(units_dataset.iloc[i]['text number'])[i])
  chatgpt_segments = ast.literal_eval(units_dataset.iloc[i]['segments'])
  total += len(total_segments)

  for k in range(len(chatgpt_segments)):
    first_token = chatgpt_segments[k].split(" ")[0]

    for j in range(len(total_segments)):
      if first_token == total_segments[j].split(" ")[0]:
        correct += 1
        break

In [2]:
correct / total

0.6516435106743477